# Agenda

1. Magic methods
2. Context managers (how objects can behave in `with` blocks)
3. Static and class methods
4. Multiple inheritance and the MRO 
5. Python object model + hierarchy
6. Properties (looks like data, acts like methods)
7. Descriptors
8. Dataclasses

# Magic methods

We can define methods on our classes, and then access them via our instances. Generally speaking, we only want a method to be invoked when we do it explicitly.